# Deep Neural Network for MNIST Classification

We'll apply all the knowledge from the lectures in this section to write a deep neural network. The problem we've chosen is referred to as the "Hello World" of deep learning because for most students it is the first deep learning algorithm they see.

The dataset is called MNIST and refers to handwritten digit recognition. You can find more about it on Yann LeCun's website (Director of AI Research, Facebook). He is one of the pioneers of what we've been talking about and of more complex approaches that are widely used today, such as covolutional neural networks (CNNs). 

The dataset provides 70,000 images (28x28 pixels) of handwritten digits (1 digit per image). 

The goal is to write an algorithm that detects which digit is written. Since there are only 10 digits (0, 1, 2, 3, 4, 5, 6, 7, 8, 9), this is a classification problem with 10 classes. 

Our goal would be to build a neural network with 2 hidden layers.

### Importing Packages

In [2]:
# !pip install tensorflow-datasets

import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

2025-01-19 13:23:24.056807: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737273204.072360    9695 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737273204.077129    9695 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-19 13:23:24.093006: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Data

In [4]:
# as_supervised splits the data into training and testing as we can use it easily
# with_info gives information about the dataset that we can use 

mnist_data, mnist_info  = tfds.load(name='mnist', with_info=True, as_supervised=True)

In [5]:
mnist_info

tfds.core.DatasetInfo(
    name='mnist',
    full_name='mnist/3.0.1',
    description="""
    The MNIST database of handwritten digits.
    """,
    homepage='http://yann.lecun.com/exdb/mnist/',
    data_dir='/home/alexender/tensorflow_datasets/mnist/3.0.1',
    file_format=tfrecord,
    download_size=11.06 MiB,
    dataset_size=21.00 MiB,
    features=FeaturesDict({
        'image': Image(shape=(28, 28, 1), dtype=uint8),
        'label': ClassLabel(shape=(), dtype=int64, num_classes=10),
    }),
    supervised_keys=('image', 'label'),
    disable_shuffling=False,
    splits={
        'test': <SplitInfo num_examples=10000, num_shards=1>,
        'train': <SplitInfo num_examples=60000, num_shards=1>,
    },
    citation="""@article{lecun2010mnist,
      title={MNIST handwritten digit database},
      author={LeCun, Yann and Cortes, Corinna and Burges, CJ},
      journal={ATT Labs [Online]. Available: http://yann.lecun.com/exdb/mnist},
      volume={2},
      year={2010}
    }""",
)

In [6]:
# We can split the train and test sets from data 

mnist_train, mnist_test = mnist_data['train'], mnist_data['test']

In [7]:
# We need validation samples so we are spliting training data using the info we have from the dataset and converting it into int64 using TF

num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

In [8]:
def scale(image, label):
    """Now we will generally scale our inputs(eg: from 0 to 1) we will write a function.
    It will take mnist image and its label as input and cast the images as float"""
    image = tf.cast(image, tf.float32)
    
    # To scale our image we have to divide it into 255 because each image consist of values from 0 to 255 in each cell in 28*28 matrix
    
    image = image/255.
    return image, label
    

In [9]:
scaled_train_and_validation_data = mnist_train.map(scale)
scaled_test_data = mnist_test.map(scale)

In [10]:
# Shuffling the data

BUFFER_SIZE = 10000
shuffled_train_and_validation_data = scaled_train_and_validation_data.shuffle(BUFFER_SIZE)

In [11]:
# Spliting validation set 

validation_data = shuffled_train_and_validation_data.take(num_validation_samples)
train_data = shuffled_train_and_validation_data.skip(num_validation_samples)

In [12]:
# Creating batch for mini batch Gradient Descent 

BATCH_SIZE = 100
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
test_data = scaled_test_data.batch(num_test_samples)

validation_inputs, validation_targets = next(iter(validation_data))

2025-01-19 13:23:56.030977: I tensorflow/core/kernels/data/tf_record_dataset_op.cc:376] The default buffer size is 262144, which is overridden by the user specified `buffer_size` of 8388608
2025-01-19 13:23:56.232090: W tensorflow/core/kernels/data/cache_dataset_ops.cc:914] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


### Model

#### Outline of the model

In [13]:
input_size = 784
output_size = 10
hidden_layer_size = 200

model = tf.keras.Sequential([
                                tf.keras.layers.Flatten(input_shape=(28,28,1)),
                                tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                                tf.keras.layers.Dense(hidden_layer_size, activation='relu'),
                                tf.keras.layers.Dense(hidden_layer_size,activation='relu'),
                                tf.keras.layers.Dense(output_size, activation='softmax'),
                            ])

/home/alexender/Desktop/Projects/My_Projects/envs/Data_Science/lib/python3.12/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


#### Choosing Optimizer and Loss Function

In [14]:
# This is a classification problem so we are choosing loss function as 'sparse_categorical_crossentropy' as its best for classification 
# For Optimizer we ara choosing the best ADAM optimizer 
# custom_optimizer = tf.keras.optimizers.Adam(learning_rate=0.0001)

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

#### Training 

In [15]:
NUM_EPOCH = 10

model.fit(train_data, epochs=NUM_EPOCH, callbacks=[tf.keras.callbacks.EarlyStopping(patience=2)], verbose=2, validation_data=(validation_inputs, validation_targets))

Epoch 1/10
540/540 - 3s - 5ms/step - accuracy: 0.9234 - loss: 0.2616 - val_accuracy: 0.9640 - val_loss: 0.1199
Epoch 2/10
540/540 - 1s - 2ms/step - accuracy: 0.9699 - loss: 0.1000 - val_accuracy: 0.9725 - val_loss: 0.0877
Epoch 3/10
540/540 - 1s - 2ms/step - accuracy: 0.9795 - loss: 0.0678 - val_accuracy: 0.9780 - val_loss: 0.0668
Epoch 4/10
540/540 - 1s - 2ms/step - accuracy: 0.9834 - loss: 0.0523 - val_accuracy: 0.9835 - val_loss: 0.0530
Epoch 5/10
540/540 - 1s - 2ms/step - accuracy: 0.9878 - loss: 0.0391 - val_accuracy: 0.9878 - val_loss: 0.0445
Epoch 6/10
540/540 - 1s - 2ms/step - accuracy: 0.9889 - loss: 0.0342 - val_accuracy: 0.9888 - val_loss: 0.0331
Epoch 7/10
540/540 - 1s - 2ms/step - accuracy: 0.9910 - loss: 0.0286 - val_accuracy: 0.9845 - val_loss: 0.0486
Epoch 8/10
540/540 - 1s - 2ms/step - accuracy: 0.9925 - loss: 0.0236 - val_accuracy: 0.9907 - val_loss: 0.0279
Epoch 9/10
540/540 - 1s - 2ms/step - accuracy: 0.9931 - loss: 0.0216 - val_accuracy: 0.9915 - val_loss: 0.0261
E

### Testing the model 

In [16]:
test_loss, test_accuracy = model.evaluate(test_data)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 152ms/step - accuracy: 0.9779 - loss: 0.0882


In [17]:
print("Test Loss: {:.2f}, Test Accuracy: {:.2f}".format(test_loss, test_accuracy*100))

Test Loss: 0.09, Test Accuracy: 97.79
